# IBM Data Science Certification - Capstone Project - Week 4
## Dataset - COVID spread in Houston/TX
### Carlos Alexandre Nako

--------------------------------------------------------------

For this analysis, the data that will be needed are the following:
- Number of COVID-19 cases by ZIP code in Houston 
- ZIP code location in Houston
- Commerce data from Foursquare

The source of these data are from a couple of sites. Bellow are the sources, brief description and the data itself.

#### Number of COVID-19 cases by ZIP code in Houston

The data for the number of COVID-19 cases will be sourced from 'City of Houston COVID-19 Data Hub'. This data is gathered by Houston Heath Department and is available to the general public. 

The link for the 'City of Houston COVID-19 Data Hub' is:

https://covid-harriscounty.hub.arcgis.com/datasets/081f346d4b9b43d88fcdc79e2e6c4517_0?geometry=-97.521%2C29.417%2C-93.349%2C30.250&showData=true

The data used is downloaded as a table ready to be used. The table has 'Total number of cases', 'Confirmed Cases', 'Active Cases', 'Recovered' and other metrics by ZIP code.

In [1]:
import pandas as pd

csv_path = 'https://opendata.arcgis.com/datasets/081f346d4b9b43d88fcdc79e2e6c4517_0.csv'
covid = pd.read_csv(csv_path)
covid.head()

,OBJECTID_1,OBJECTID,ZIP,POSTAL,STATE,DATE_MOD,code,ZIP_TYPE,Shape_STArea_1,Shape_STLength_1,...,TotalConfirmedCases,ActiveCases,ActiveCases_str,Recovered,Death,Death_str,EditDate,Today,Shape__Area,Shape__Length
0,1,229,77003,HOUSTON,TX,2011/10/07 00:00:00+00,HA,STANDARD,7.419092e+07,57485.331546,...,1122,14,14,1091,17,17,2021/05/20,2021/05/20,0.000643,0.170022
1,2,223,77004,HOUSTON,TX,2011/10/12 00:00:00+00,HA,STANDARD,1.651628e+08,67522.582037,...,2048,32,32,1983,33,33,2021/05/20,2021/05/20,0.001431,0.199602
2,3,225,77006,HOUSTON,TX,2011/06/08 00:00:00+00,HA,STANDARD,6.314081e+07,43137.266443,...,1386,13,13,1366,7,7,2021/05/20,2021/05/20,0.000547,0.127231
3,4,218,77007,HOUSTON,TX,2011/06/09 00:00:00+00,HA,STANDARD,2.188591e+08,88174.369103,...,3091,51,51,3031,9,9,2021/05/20,2021/05/20,0.001897,0.264630
4,5,86,77013,HOUSTON,TX,2011/06/17 00:00:00+00,HA,STANDARD,2.571837e+08,94063.832847,...,1623,34,34,1574,15,15,2021/05/20,2021/05/20,0.002230,0.279543


#### ZIP code location in Houston

The data for the ZIP code location will be sourced from Zip Atlas which is a company specialized in Zip code for US.

The link for Zip Atlas is:

http://zipatlas.com/us/tx/houston/zip-code-comparison/population-density.htm

The data will be retrieved with BeautifulSoup directly form the website.

In [4]:
import requests
from bs4 import BeautifulSoup

html_data = requests.get('http://zipatlas.com/us/tx/houston/zip-code-comparison/population-density.htm')
beautiful_soup = BeautifulSoup(html_data.content, 'html.parser')
table_contents=[]
table=beautiful_soup.find_all('table')[13]

for row in table.findAll('tr'):
    td = row.find_all('td')
    rows = [row.text for row in td]
    table_contents.append(rows)

df=pd.DataFrame(table_contents)

new_header = df.iloc[0] 
df = df[1:] 
df.columns = new_header 

df[['Latitude','Longitude']] = df['Location'].str.split(',',expand=True)
df = df.drop(columns = ['#','Location','People / Sq. Mile','National Rank'])

df['Zip Code'] = df['Zip Code'].astype('int64')
df['Population'] = df['Population'].str.replace(',','')
df['Population'] = df['Population'].astype('int64')

df

,Zip Code,City,Population,Latitude,Longitude
1,77046,"Houston, Texas",471,29.733084,-95.430659
2,77081,"Houston, Texas",49691,29.712099,-95.480935
3,77036,"Houston, Texas",76146,29.699048,-95.536507
4,77006,"Houston, Texas",18875,29.741003,-95.391271
5,77057,"Houston, Texas",35491,29.744068,-95.489210
...,...,...,...,...,...
92,77049,"Houston, Texas",16598,29.832928,-95.149040
93,77094,"Houston, Texas",7779,29.769285,-95.681292
94,77050,"Houston, Texas",4357,29.903351,-95.262070
95,77032,"Houston, Texas",10796,29.987805,-95.353412


#### Commerce data form Foursquare

Business data comes from Foursquare. To do that, ZIP location data will be used to retrive the Foursquare data. After retrieving business data for each ZIP code, they will be combined into one single table.

The Foursquare will be retrived as json and stored as table similar to what was done on the previous week.

In [5]:
import json
from pandas.io.json import json_normalize

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

*Setting up Foursquare credentials

In [9]:
# The code was removed by Watson Studio for sharing.

Credentails Set: CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN


In [8]:
zip_size = df.shape[0]
nearby_venues = pd.DataFrame()
venue_list = pd.DataFrame()
no_venue = pd.DataFrame(columns=['No ZIP'])

radius = 1600
LIMIT = 500

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

for i in range(0,zip_size):

    PostalCode_latitude = df.loc[i+1, 'Latitude'] # Postal Code latitude value
    PostalCode_longitude = df.loc[i+1, 'Longitude'] # Postal Code longitude value
    PostalCode_ZIP = df.loc[i+1, 'Zip Code'] # Postal Code value

    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, PostalCode_latitude, PostalCode_longitude, VERSION, radius, LIMIT)
    results = requests.get(url).json()
    
    venues = results['response']['groups'][0]['items']
    
    nearby_venues = nearby_venues.iloc[0:0]
    nearby_venues = json_normalize(venues) # flatten JSON
    
    if len(nearby_venues) == 0:
        no_venue = no_venue.append({'No ZIP' : PostalCode_ZIP}, ignore_index=True)
    else:
        nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
        nearby_venues = nearby_venues.loc[:, filtered_columns]
        nearby_venues['PostalCode'] = PostalCode_ZIP
    
        venue_list = venue_list.append(nearby_venues, ignore_index=True)

venue_list.columns = [col.split(".")[-1] for col in venue_list.columns]
venue_list

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng,PostalCode
0,Eunice,Seafood Restaurant,29.732910,-95.428648,77046
1,Levy Park,Park,29.732945,-95.424319,77046
2,Urban Harvest Farmers Market,Farmers Market,29.735229,-95.424442,77046
3,Game Over Videogames,Video Game Store,29.728696,-95.431958,77046
4,100% Taquito,Mexican Restaurant,29.728784,-95.431001,77046
...,...,...,...,...,...
4339,AB Books,Bookstore,29.896336,-95.175333,77044
4340,Shy Girls Saloon,Bar,29.884797,-95.161700,77044
4341,Garrett Road Ice House,Dive Bar,29.885035,-95.161340,77044
4342,Sheldon Aqueduct,River,29.888402,-95.158141,77044
